### Food Image Classification With a CNN

In [1]:
# load the necessary packages
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd

import os
import numpy as np
import matplotlib.pyplot as plt

### Load and Preprocess Data for Model

In [2]:
batch_size = 150
epochs = 15
IMG_HEIGHT = 180
IMG_WIDTH = 180
train_dir = 'data/resized/train'
test_dir = 'data/resized/test'
dropout_rate = 0.15

In [3]:
train_image_generator = ImageDataGenerator(rescale=1./255)
test_image_generator = ImageDataGenerator(rescale=1./255)

In [4]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                          directory=train_dir,
                                                          shuffle=True,
                                                          target_size=(IMG_HEIGHT,IMG_WIDTH))

Found 75750 images belonging to 101 classes.


In [5]:
test_data_gen = test_image_generator.flow_from_directory(batch_size=batch_size,
                                                          directory=test_dir,
                                                          shuffle=True,
                                                          target_size=(IMG_HEIGHT,IMG_WIDTH))

Found 25250 images belonging to 101 classes.


In [6]:
model = Sequential([
    Conv2D(64, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
    MaxPooling2D(),
    BatchNormalization(),
    Dropout(rate=dropout_rate),
    Conv2D(128, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    BatchNormalization(),
    Dropout(rate=dropout_rate),
    Conv2D(256, 5, padding='same', activation='relu'),
    MaxPooling2D(),
    BatchNormalization(),
    Dropout(rate=dropout_rate),
    Conv2D(512, 5, padding='same', activation='relu'),
    MaxPooling2D(),
    BatchNormalization(),
    Dropout(rate=dropout_rate),
    Flatten(),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dense(1024, activation='relu'),
    BatchNormalization(),
    Dense(101,activation=None)
])

In [7]:
model.compile(optimizer='adam',
              loss='cosine_similarity',
              metrics=['accuracy'])

In [8]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "checkpoints/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=5)

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 180, 180, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 90, 90, 64)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 90, 90, 64)        256       
_________________________________________________________________
dropout (Dropout)            (None, 90, 90, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 90, 90, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 45, 45, 128)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 45, 45, 128)       5

In [10]:
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=75750//batch_size,
    epochs=epochs,
    validation_data=test_data_gen,
    validation_steps=25250//batch_size,
    workers=1,
    use_multiprocessing=False,
    callbacks=[cp_callback]
)

Epoch 1/15

Epoch 00001: saving model to checkpoints/cp-0001.ckpt
  1/505 [..............................] - ETA: 46:25 - loss: 0.0094 - accuracy: 0.0200
Epoch 00001: saving model to checkpoints/cp-0001.ckpt
  2/505 [..............................] - ETA: 30:54 - loss: 0.0099 - accuracy: 0.0133
Epoch 00001: saving model to checkpoints/cp-0001.ckpt
  3/505 [..............................] - ETA: 26:13 - loss: 0.0088 - accuracy: 0.0089
Epoch 00001: saving model to checkpoints/cp-0001.ckpt
  4/505 [..............................] - ETA: 23:29 - loss: 0.0054 - accuracy: 0.0067
Epoch 00001: saving model to checkpoints/cp-0001.ckpt
  5/505 [..............................] - ETA: 21:47 - loss: 0.0024 - accuracy: 0.0107
Epoch 00001: saving model to checkpoints/cp-0001.ckpt
  6/505 [..............................] - ETA: 20:39 - loss: -0.0011 - accuracy: 0.0100
Epoch 00001: saving model to checkpoints/cp-0001.ckpt
  7/505 [..............................] - ETA: 19:51 - loss: -0.0023 - accuracy:

 58/505 [==>...........................] - ETA: 14:42 - loss: -0.0574 - accuracy: 0.0352
Epoch 00001: saving model to checkpoints/cp-0001.ckpt
 59/505 [==>...........................] - ETA: 14:39 - loss: -0.0583 - accuracy: 0.0366
Epoch 00001: saving model to checkpoints/cp-0001.ckpt
 60/505 [==>...........................] - ETA: 14:38 - loss: -0.0590 - accuracy: 0.0367
Epoch 00001: saving model to checkpoints/cp-0001.ckpt
 61/505 [==>...........................] - ETA: 14:36 - loss: -0.0594 - accuracy: 0.0368
Epoch 00001: saving model to checkpoints/cp-0001.ckpt
 62/505 [==>...........................] - ETA: 14:35 - loss: -0.0598 - accuracy: 0.0368
Epoch 00001: saving model to checkpoints/cp-0001.ckpt
 63/505 [==>...........................] - ETA: 14:33 - loss: -0.0603 - accuracy: 0.0371
Epoch 00001: saving model to checkpoints/cp-0001.ckpt
 64/505 [==>...........................] - ETA: 14:31 - loss: -0.0609 - accuracy: 0.0372
Epoch 00001: saving model to checkpoints/cp-0001.ckpt

KeyboardInterrupt: 

In [ ]:
if not os.path.isdir('./tmp/models'):
    os.makedirs('./tmp/models')
model.save('tmp/models/CNN_MODEL_V1.h5')